In [1]:
#import tools 
import requests 
from bs4 import BeautifulSoup
import re
import bs4

In [2]:
r = requests.get("https://www.switchscores.com/top-rated/all-time") #set up url

In [3]:
r.text          
demo =r.text 

In [4]:
soup = BeautifulSoup(demo, "html.parser")

In [5]:
for link in soup.find_all('a'):
    print(link.get('href'))   ## get all links in the web for a

https://www.switchscores.com
https://www.switchscores.com
#
https://www.switchscores.com/games
https://www.switchscores.com/games/by-title
https://www.switchscores.com/games/by-date
https://www.switchscores.com/games/by-category
https://www.switchscores.com/games/by-tag
https://www.switchscores.com/games/by-series
https://www.switchscores.com/games/by-collection
#
https://www.switchscores.com/top-rated
https://www.switchscores.com/top-rated/all-time
https://www.switchscores.com/top-rated/by-year/2022
https://www.switchscores.com/top-rated/by-year/2021
https://www.switchscores.com/top-rated/by-year/2020
https://www.switchscores.com/top-rated/by-year/2019
https://www.switchscores.com/top-rated/by-year/2018
https://www.switchscores.com/top-rated/by-year/2017
https://www.switchscores.com/top-rated/multiplayer
#
https://www.switchscores.com/lists
https://www.switchscores.com/games/recent
https://www.switchscores.com/games/upcoming
https://www.switchscores.com/games/on-sale
https://www.switc

In [6]:
def getHTMLText(url):
    try:
        r =requests.get(url,timeout = 30)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return ""